# Create perturbed DCM files for Robert to analyze with traditional planning systems

In [1]:
import sys
import time
import glob
import os

import numpy as np
import skimage.morphology as skm
import matplotlib.pyplot as plt

import pydicom
from DicomRTTool.ReaderWriter import DicomReaderWriter
import SimpleITK as sitk

In [2]:
def find_boundary_points(volume):
    """
    Find points on the boundary of a region of interest.
    These points will then be used to create perturbations.
    """
    ball = skm.ball(3)
    volume_larger = skm.binary_dilation(volume[:, :, :], ball)
    boundary_volume = volume_larger - volume[:, :, :]
    points = np.nonzero(boundary_volume)
    out_points = []
    for idx in range(len(points[0])):
        x = points[0][idx]
        y = points[1][idx]
        z = points[2][idx]
        out_points.append([x, y, z])
    return out_points


def dilate_at(volume, point):
    """
    Dilate the binary volume 'volume' at the point specified bt point.
    """
    ball = skm.ball(3)
    point_vol = np.zeros(volume[:, :, :].shape, dtype=np.uint8)
    point_vol[point[0], point[1], point[2]] = 1
    volume_out = skm.binary_dilation(point_vol, ball).astype(np.uint8)
    volume_out += volume[:, :, :].astype(np.uint8)
    volume_out[volume_out >= 1] = 1
    volume_out = volume_out[:, :, :]
    return volume_out

In [3]:
base_input_path = "/home/akamath/Documents/deep-planner/data/raw/"
base_output_path = "/home/akamath/Documents/deep-planner/data/dcm/"
os.makedirs(base_output_path, exist_ok=True)

n_subjects = 3

for subject_id in range(81, 81 + n_subjects + 1):

    str_id =  str(subject_id).zfill(3)
    subject_name = "DLDP_" + str_id
    
    print("Analyzing subject: ", subject_name)

    this_subject = os.path.join(base_input_path, subject_name)
    this_subject_out = os.path.join(base_output_path, subject_name)

    Dicom_reader = DicomReaderWriter(description='Examples', arg_max=True)
    Dicom_reader.walk_through_folders(this_subject) # need to define in order to use all_roi method
    all_rois = Dicom_reader.return_rois(print_rois=True)  # Return a list of all rois present, and print them

    Contour_Names = ['Brainstem'] 
    # Associations work as {'variant_name': 'desired_name'}
    associations = {'r_brainstem': 'Brainstem'}

    Dicom_reader.set_contour_names_and_associations(Contour_Names=Contour_Names, associations=associations)

    indexes = Dicom_reader.which_indexes_have_all_rois()  # Check to see which indexes have all of the rois we want, now we can see indexes

    pt_indx = indexes[-1]
    Dicom_reader.set_index(pt_indx)  # This index has all the structures, corresponds to pre-RT T1-w image for patient 011
    Dicom_reader.get_images_and_mask()  # Load up the images and mask for the requested index

    image = Dicom_reader.ArrayDicom # image array
    mask = Dicom_reader.mask # mask array
    dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
    mask_sitk_handle = Dicom_reader.annotation_handle # SimpleITK mask handle

    out_volume = np.zeros(image.shape + (2,))
    out_volume[:, :, :, 1] = mask
    Dicom_reader.prediction_array_to_RT(prediction_array=out_volume, output_dir=this_subject_out,
                                        ROI_Names=['brainstem_original'])

    # Make perturbations here, and save to DCM again.
    point_set = find_boundary_points(mask)
    # At this stage, do perturbation on the OAR boundary.
    
    for idx in list(np.linspace(0, len(point_set) - 1, num=10).astype(np.int16)):
        mask_dilated = dilate_at(mask, point_set[idx])
        out_volume = np.zeros(image.shape + (2,))
        out_volume[:, :, :, 1] = mask_dilated
        Dicom_reader.prediction_array_to_RT(prediction_array=out_volume, output_dir=this_subject_out,
                                            ROI_Names=['brainstem_' + str(idx)])

Loading through DICOM files: 100%|██████████| 1/1 [00:00<00:00, 21.88it/s]

Analyzing subject:  DLDP_081
Loading from /home/akamath/Documents/deep-planner/data/raw/DLDP_081
Compiling dictionaries together...
Index 0, description GBM  at /home/akamath/Documents/deep-planner/data/raw/DLDP_081
1 unique series IDs were found. Default is index 0, to change use set_index(index)
The following ROIs were found
r_brainstem
r_cochlea_l
r_cochlea_r
r_eye_l
r_eye_r
r_hippocampus_l
r_hippocampus_r
r_lacrimal_l
r_lacrimal_r
r_opticchiasm
ptv
r_opticenerve_l
r_opticnerve_r
r_pituitary
brain
body
The following indexes have all ROIs present
Index 0, located at /home/akamath/Documents/deep-planner/data/raw/DLDP_081
Finished listing present indexes
Loading images for GBM  at 
 /home/akamath/Documents/deep-planner/data/raw/DLDP_081



Running off a template
Writing data for brainstem_original
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_0
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_2276
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_4552
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_6829
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_9105
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Writing data for brainstem_11382
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Running off a template
Wr

Loading through DICOM files: 100%|██████████| 1/1 [00:00<00:00, 20.28it/s]

Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_081
Finished!
Analyzing subject:  DLDP_082
Loading from /home/akamath/Documents/deep-planner/data/raw/DLDP_082
Compiling dictionaries together...
Index 0, description None at /home/akamath/Documents/deep-planner/data/raw/DLDP_082
1 unique series IDs were found. Default is index 0, to change use set_index(index)
The following ROIs were found
cochlea_l
cochlea_r
hippocampus_l
hippocampus_r
lacrimal_l
lacrimal_r
opticchiasm
opticenerve_l
opticnerve_r
pituitary
ptv_high
brain
body
eye_l
brainstem
eye_r
The following indexes have all ROIs present
Index 0, located at /home/akamath/Documents/deep-planner/data/raw/DLDP_082
Finished listing present indexes
Loading images for None at 
 /home/akamath/Documents/deep-planner/data/raw/DLDP_082



...

Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_082
Finished!
Analyzing subject:  DLDP_083
Loading from /home/akamath/Documents/deep-planner/data/raw/DLDP_083
Compiling dictionaries together...
Index 0, description None at /home/akamath/Documents/deep-planner/data/raw/DLDP_083
1 unique series IDs were found. Default is index 0, to change use set_index(index)
The following ROIs were found
r_hippocampus_r
r_lacrimal_l
r_lacrimal_r
r_opticchiasm
r_opticenerve_l
r_eye_r
r_opticnerve_r
r_pituitary
brain
body
r_hippocampus_l
r_eye_l
r_cochlea_r
r_cochlea_l
r_brainstem
ptv
The following indexes have all ROIs present
Index 0, located at /home/akamath/Documents/deep-planner/data/raw/DLDP_083
Finished listing present indexes
Loading images for None at 
 /home/akamath/Documents/deep-planner/data/raw/DLDP_083



Running off a template
Writing data for brainstem_original
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_0
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_2437
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_4875
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_7313
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_9751
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Writing data for brainstem_12189
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Running off a template
Wr

Loading through DICOM files: 100%|██████████| 1/1 [00:00<00:00, 22.29it/s]

Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_083
Finished!
Analyzing subject:  DLDP_084
Loading from /home/akamath/Documents/deep-planner/data/raw/DLDP_084
Compiling dictionaries together...
Index 0, description None at /home/akamath/Documents/deep-planner/data/raw/DLDP_084
1 unique series IDs were found. Default is index 0, to change use set_index(index)
The following ROIs were found
ptv_high
pituitary
opticnerve_r
opticenerve_l
opticchiasm
lacrimal_r
lacrimal_l
hippocampus_r
hippocampus_l
eye_r
eye_l
brainstem
cochlea_l
cochlea_r
brain
body
The following indexes have all ROIs present
Index 0, located at /home/akamath/Documents/deep-planner/data/raw/DLDP_084
Finished listing present indexes
Loading images for None at 
 /home/akamath/Documents/deep-planner/data/raw/DLDP_084



Running off a template
Writing data for brainstem_original
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_0
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_2018
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_4036
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_6054
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_8072
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Writing data for brainstem_10091
Writing out data.../home/akamath/Documents/deep-planner/data/dcm/DLDP_084
Finished!
Running off a template
Wr